In [1]:
#importing libraries

In [1]:
import numpy as np
import tensorflow as tf
import re #clean the text, to simplyfy conversation
import time #to measure training time

In [2]:
tf.__version__

AttributeError: module 'tensorflow' has no attribute '__version__'

In [3]:
tf.InteractiveSession()

AttributeError: module 'tensorflow' has no attribute 'InteractiveSession'

In [116]:
#part 1 - data preprocessing

In [117]:
#importing dataset

#import os

#path ='/home/shubh/Desktop/chatbot dataset'

#file = 'movie_lines.txt'

#lines = open(os.path.join(path,file))

lines = open('/home/shubh/Desktop/chatbot dataset/movie_lines.txt', encoding = 'utf-8', errors = "ignore").read().split('\n') 
conversations = open('/home/shubh/Desktop/chatbot dataset/movie_conversations.txt', encoding = 'utf-8', errors = "ignore").read().split('\n')

In [118]:
# here in conversation and in lines -represents-> u- user, & m- movies following to their conversations 

In [119]:
#conversations

In [120]:
#lines

In [121]:
# creating a dictionary that maps each line and its id

id2line = {}
for line in lines :
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [122]:
#id2line

In [123]:
#line

In [124]:
#creating the list of all conversations

conversations_ids = []
for conversation in conversations [ :-1]:
    _conversation = conversation.split (' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [125]:
#conversations_ids

In [126]:
# getting separately questions and answers

questions = [ ]
answers = [ ]
for conversation in conversations_ids : 
    for i in range (len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [127]:
#questions

In [128]:
#answers

In [129]:
# first cleaning of text

def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r"\"ll","will",text)
    text = re.sub(r"\"ve","have",text)
    text = re.sub(r"\"d","would",text)
    text = re.sub(r"didn't","did not",text)
    text = re.sub(r"[-()\"#*$%&!~|,.?{}+:;<>@/^]","",text)
    return text

In [130]:
#cleaning the questions

clean_questions = []
for question in questions:
    clean_questions.append (clean_text(question))

In [131]:
# cleaning the answers

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [132]:
#clean_questions

In [133]:
#clean_answers

In [134]:
# creating a dictionary that maps each word to its occurance in questions and in answers


word2count = {}  #making word2count dictionary

#for questions

for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count [word] = 1
        else:
            word2count [word] += 1
            
            
#now for answers-
for answer in clean_answers:
    for word in answer:
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
        
        

In [135]:
# creating two dictionaries that maps the questions word and answers word to a unique integer
 
    
    #here we are taking a threshold of word occurance in questions and  in answers
  #  if any word is coming below threshold quantity we will exclude that and whose word
   # qunatity is greater than the threshold we will take that word.
    
threshold = 20

In [136]:
#making dictionary for questions

questionsword2int = {}
word_number = 0 #again we are starting from zero
for word,count in word2count.items():
    if count >= threshold:
        questionsword2int[word] = word_number
        word_number += 1
#making dictionary for answers
answersword2int = {}
word_number = 0
for word,count in word2count.items():
    if count >= threshold:
        answersword2int[word] = word_number
        word_number += 1
        
            


In [137]:
# adding the last tokens to the directories

tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]

for token in tokens:
    questionsword2int[token] = len(questionsword2int) +1
    
for token in tokens:
    answersword2int[token] = len(answersword2int) +1

In [138]:
# creating the inverse dictionary for answersword2int dictionary to map the words
# w - word & w_i - word integer
answersints2word = {w_i:w for w,w_i in answersword2int.items()}

In [139]:
#answersints2word

In [140]:
# adding the end of string token to the end of evry answers
# doing beacause EOS needed in the end of decoding in seq2seq model


for i in range (len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [141]:
#clean_answers

In [142]:
# translating all the question and answers in to integer
#and replacing all the words that were filtered out by <OUT>
# to sort all the questions and answers by their length to optimize training performance

In [143]:
#list for changing the words of each question into integer value

questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionsword2int:
            ints.append(questionsword2int['<OUT>'])
        else:
            ints.append(questionsword2int[word])
    questions_into_int.append(ints)

In [144]:
#list for changing the words of each answer into integer value

answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in questionsword2int:
            ints.append(questionsword2int['<OUT>'])
        else:
            ints.append(questionsword2int[word])
    answers_into_int.append(ints)

In [145]:
#answers_into_int

In [146]:
#questions_into_int

In [147]:
#sorting question and answers by the length of question
#it will reduce the padding during the training and speedup our training


sorted_clean_questions = []
sorted_clean_answers = []
for length in range (1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1])==length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [148]:
#sorted_clean_questions

In [149]:
#sorted_clean_answers


In [150]:
#Creating placeholders for the inputs and targets

In [151]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = "input")#create placeholderr for inputs
    targets = tf.placeholder(tf.int32,[None,None], name  = "target")#create placeholder for targets
# making parameter(hyperparameter[learning_rate & keep_prob]) used to control dropout rate
# dropouts are the rate of neurons that we are choosed to overwrite during one iteration in training
# we are using 20% dropout to deactivate 20% neuron
# we increase or decrease dropout by using keep_prob hyperparameter
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

In [152]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0],[batch_size, -1],[1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets
    #batch_size is no. of lines in batch
    #strided_slice - extract the subset of tensors
    # for horizontal, axis=1 & for vertical, axis=0 

In [153]:
# creating the encoder rnn layer

In [154]:
def encoder_rnn (rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]*num_layers)
    # num_layers create encoder cell
    # encoder cell consist of several lstm layers
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                       cell_bw = encoder_cell,
                                                       sequence_length = sequence_length,
                                                       inputs = rnn_inputs,
                                                       dtype = tf.float32)
    # bidirectional_dynamic_rnn take input and build independent forward and backward rnn(input of forward and backward must match)
    # _, -indicates that we only want second element returned by this future bidirectional dinemic rnn function
    return encoder_state

In [155]:
#for encoder we only need encoder state and for decoder we only need decoder output
# decoding the training set


In [156]:
def decode_training_set(encoder_state,
                        decoder_cell,
                        decoder_embedded_input,
                        sequence_length,
                        decoding_scope,
                        output_function,
                        keep_probe,
                        batch_size):
#decoder getting encoder_state as a part of input proceed to decoding
#decoder_cell - cell in the rnn of the decoder
#embeddings are useful to reduce the dimensoility of categorical variables
#embeddings - is a mapping from descrete objects(i.e. words) to vectors of real no.s(uniquiley) 
#here we roviding i/p to embeddings
#decoding_scope - advance data structure that wraps TF variables  

    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    #we are processing 3 dimn matrix
    
    #preprocesses the training data
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    # attention_keys  = key to compare with target states
    #attention_values = values to construct the context vector(return by encoder and used by decoder)
    #attention_score_function - used to compute similarity between keys and target_states
    # attentio construct_function - used to built attention state
    
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                             attention_keys,
                                                                             attention_values,
                                                                             attention_score_function,
                                                                             attention_construct_function,
                                                                             name = "attn_dec_train")#this is the name scope for the decoder function
    
    
    decoder_output, decoder_final_state, decoder_final_context_state = tf.dynamic_rnn_decoder(decoder_cell,
                                                                                             training_decoder_function,
                                                                                             decoder_embedded_input,
                                                                                             sequence_length,
                                                                                             scope = decoding_scope)
    #now we take the droput of the decoder_output
    
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_probe)
    return output_function(decoder_output_dropout)

In [160]:
def decode_test_set(encoder_state,
                    decoder_cell,
                    decoder_embeddings_matrix,
                    sos_id,
                    eos_id,
                    maximum_length,
                    num_words,
                    sequence_length,
                    decoding_scope,
                    output_function,
                    keep_probe,
                    batch_size):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    
    
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")#this is the name scope for the decoder function
    
    test_predictions, decoder_final_state, decoder_final_context_state = tf.dynamic_rnn_decoder(decoder_cell,
                                                                                              test_decoder_function,
                                                                                              scope = decoding_scope)
                                                                                              
                                                                                              
                                                                                              
                                                                                            
                                                                  
    return test_predictions
    

In [ ]:
def decoder_rnn(decoder_embedded_input,
                decoder_embeddings_matrix,
                encoder_state,
                num_words,
                sequence_length,
                rnn_size,
                num_layers,
                word2int,
                keep_prob,
                batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        #create lstm layer
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        #now apply some dropout regulariztion to reduce overfitting and increase the accuracy
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_probe = keep_probe)
        